In [88]:
# added because the logging was annoying
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)
from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten, Concatenate
from keras.layers import Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam
import psycopg2
import numpy as np
from sklearn.utils import shuffle

In [89]:
file = open('connect.txt', 'r')
connection_string = file.readline()
conn = psycopg2.connect(connection_string)
curr = conn.cursor()

# user count 
curr.execute('SELECT COUNT(*) FROM users')
num_users = curr.fetchone()

# house count
curr.execute('SELECT COUNT(*) FROM houses')
num_houses = curr.fetchone()

# users, houses, scores
curr.execute('SELECT user_id, house_id, actual_score from matches')
data = curr.fetchall()
shuffle(data, random_state=40)
data = [list(d) for d in zip(*data)]
users, houses, matches = data[0], data[1], data[2]

In [90]:
'''
Temporary method to handle database key issue
'''
def remapUsers(temp):
    items = [7, 8, 12, 13, 14, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30]
    new_users =[]
    for t in temp: 
        for i,v in enumerate(items):
            if t == v:
                new_users.append(i)
    return new_users
users = remapUsers(users)

In [113]:
K = 10
u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(num_users[0], K, embeddings_regularizer=l2(0))(u) 
m_embedding = Embedding(num_houses[0], K, embeddings_regularizer=l2(0))(m)
u_bias = Embedding(num_users[0], 1, embeddings_regularizer=l2(0))(u) 
m_bias = Embedding(num_houses[0], 1, embeddings_regularizer=l2(0))(m) 
x = Dot(axes=2)([u_embedding, m_embedding]) 

In [119]:
x = Add()([x, u_bias, m_bias])
x = Flatten()(x) # (N, 1)

model = Model(inputs=[u, m], outputs=x)
model.compile(
  loss='mse',
  # optimizer='adam',
  # optimizer=Adam(lr=0.01),
  optimizer=SGD(lr=0.08, momentum=0.9),
  metrics=['accuracy'],
)

r = model.fit(
  x=[users[200:], houses[200:]],
  y=matches[200:],
  epochs=20,
  batch_size=128,
  validation_data=(
    [users[:100], houses[:100]], 
      matches[:100]
  )
)

Train on 2265 samples, validate on 100 samples
Epoch 1/20
2265/2265 [==============================] - 0s 151us/step - loss: 0.1195 - accuracy: 1.0000 - val_loss: 0.3942 - val_accuracy: 1.0000
Epoch 2/20
2265/2265 [==============================] - 0s 18us/step - loss: 0.0484 - accuracy: 1.0000 - val_loss: 0.3830 - val_accuracy: 1.0000
Epoch 3/20
2265/2265 [==============================] - 0s 19us/step - loss: 0.0295 - accuracy: 1.0000 - val_loss: 0.3775 - val_accuracy: 1.0000
Epoch 4/20
2265/2265 [==============================] - 0s 16us/step - loss: 0.0227 - accuracy: 1.0000 - val_loss: 0.3949 - val_accuracy: 1.0000
Epoch 5/20
2265/2265 [==============================] - 0s 20us/step - loss: 0.0195 - accuracy: 1.0000 - val_loss: 0.3955 - val_accuracy: 1.0000
Epoch 6/20
2265/2265 [==============================] - 0s 20us/step - loss: 0.0176 - accuracy: 1.0000 - val_loss: 0.3923 - val_accuracy: 1.0000
Epoch 7/20
2265/2265 [==============================] - 0s 19us/step - loss: 0.016

In [120]:
def makePredictionsAndComputeAccuracy(test_X, test_y):
    res = model.predict(test_X)
    res = [ 1 if r >= 0 else 0 for r in res ]
    print(len([i for i, j in zip(test_y, res) if i == j])/len(test_y))

In [121]:
makePredictionsAndComputeAccuracy([users[100:500], houses[100:500]], matches[100:500])

0.77


In [ ]:
# the neural network
x = Dense(400)(x)
# x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(100)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
x = Dense(1)(x)

In [ ]:
model = Model(inputs=[u, m], outputs=x)
model.compile(
  loss='mse',
  # optimizer='adam',
  # optimizer=Adam(lr=0.01),
  optimizer=SGD(lr=0.08, momentum=0.9),
  metrics=['mse'],
)

In [ ]:
r = model.fit(
  x=[np.array(users[:10]), np.array(houses[:10])],
  y=np.array(matches[:10]),
  epochs=15,
  batch_size=10,
  validation_data=(
    [np.array(users[10:]), np.array(houses[1000:])],
    np.array(matches[1000:])
  )
  )

In [ ]:
conn = databaseConnection()